In [30]:
import requests
import json
import pandas as pd

In [ ]:
API_KEY = "здесь был токен"  # Ключ API YouTube
CHANNEL_HANDLE = "matroskin_perm"  # Имя канала 

In [ ]:
# Функция для получения ID плейлиста загрузок (uploads) канала по его имени
def get_uploads_playlist_id(api_key, channel_handle): #
    url = "https://www.googleapis.com/youtube/v3/channels"
    params = {
        "part": "contentDetails",
        "forHandle": channel_handle,
        "key": api_key
    }
    # Отправляем GET-запрос к API YouTube
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    items = data.get("items", [])
    if not items:
        raise ValueError("Канал с таким handle не найден")
    # ID плейлиста всех загрузок канала содержится в contentDetails.relatedPlaylists.uploads
    uploads_playlist_id = items[0]["contentDetails"]["relatedPlaylists"]["uploads"]
    return uploads_playlist_id

In [ ]:
if __name__ == "__main__": #
    try:
        uploads_playlist_id = get_uploads_playlist_id(API_KEY, CHANNEL_HANDLE)
        print("ID плейлиста загрузок (uploads):", uploads_playlist_id)
    except Exception as e:
        print("Ошибка:", e)

ID плейлиста загрузок (uploads): UUq7EZGxyVNbYtFSSCqtA2Kw


In [ ]:
UPLOADS_PLAYLIST_ID = "UUq7EZGxyVNbYtFSSCqtA2Kw"

In [ ]:
# Функция для получения всех ID видео из плейлиста загрузок
def get_all_video_ids(api_key, playlist_id):
    video_ids = []
    base_url = "https://www.googleapis.com/youtube/v3/playlistItems"
    params = {
        "part": "snippet",
        "playlistId": playlist_id,
        "maxResults": 50,
        "key": api_key
    }
    while True:
        # Запрашиваем данные плейлиста с видео
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        # Выделяем ID каждого видео в ответе
        for item in data.get("items", []):
            video_id = item["snippet"]["resourceId"]["videoId"]
            video_ids.append(video_id)
        # Проверка наличия следующей страницы результатов
        if "nextPageToken" in data:
            params["pageToken"] = data["nextPageToken"]
        else:
            break
    return video_ids

In [ ]:
# Функция для получения детальной информации о видео по списку ID
def get_videos_details(api_key, video_ids):
    video_details = []
    base_url = "https://www.googleapis.com/youtube/v3/videos"
    for i in range(0, len(video_ids), 50):
        batch_ids = video_ids[i:i+50]
        params = {
            "part": "snippet,contentDetails,statistics",
            "id": ",".join(batch_ids),
            "key": api_key
        }
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        video_details.extend(data.get("items", []))
    return video_details

In [ ]:
# Упрощенная функция для приведения вложенных значений к удобному формату (строке JSON)
def flatten_value(value):
    if isinstance(value, (dict, list)):
        return json.dumps(value, ensure_ascii=False)
    return value

In [ ]:
# Функция преобразования списка видео-словарей в датафрейм pandas
def build_full_dataframe(videos):
    rows = []
    for v in videos:
        row = {}
        for key, val in v.items():
            row[key] = flatten_value(val)
        rows.append(row)
    df = pd.DataFrame(rows)
    return df

In [ ]:
# Главный блок запуска кода
if __name__ == "__main__":
    video_ids = get_all_video_ids(API_KEY, UPLOADS_PLAYLIST_ID)
    videos = get_videos_details(API_KEY, video_ids)
    df = build_full_dataframe(videos)
    print(df.head())

            kind                         etag           id  \
0  youtube#video  6yQH30wbdu77dEHPZLfVvgQK0W8  TK8G3ZFLGIY   
1  youtube#video  gHbtVDarmvLMD0bb3yEgDxV2A6k  ibmF0emEK0s   
2  youtube#video  O5a6Idne7JJ0o5YTleqr4xT-LVw  Hd74vM8tTmI   
3  youtube#video  rjFP7L6GaGo6Gxi0CFvy4O3Ip6M  Gq9SZWmkENg   
4  youtube#video  WaFZE35Ow75MNZnxk_qx_yLhJYM  838e8voBnFw   

                                             snippet  \
0  {"publishedAt": "2025-07-15T12:30:58Z", "chann...   
1  {"publishedAt": "2025-07-05T19:00:41Z", "chann...   
2  {"publishedAt": "2025-05-30T19:00:24Z", "chann...   
3  {"publishedAt": "2025-03-08T19:00:11Z", "chann...   
4  {"publishedAt": "2025-02-22T19:00:04Z", "chann...   

                                      contentDetails  \
0  {"duration": "PT15M1S", "dimension": "2d", "de...   
1  {"duration": "PT15M9S", "dimension": "2d", "de...   
2  {"duration": "PT31M14S", "dimension": "2d", "d...   
3  {"duration": "PT32M35S", "dimension": "2d", "d...   
4  {"durat

In [46]:
df

,kind,etag,id,snippet,contentDetails,statistics
0,youtube#video,6yQH30wbdu77dEHPZLfVvgQK0W8,TK8G3ZFLGIY,"{""publishedAt"": ""2025-07-15T12:30:58Z"", ""chann...","{""duration"": ""PT15M1S"", ""dimension"": ""2d"", ""de...","{""viewCount"": ""24"", ""likeCount"": ""4"", ""favorit..."
1,youtube#video,gHbtVDarmvLMD0bb3yEgDxV2A6k,ibmF0emEK0s,"{""publishedAt"": ""2025-07-05T19:00:41Z"", ""chann...","{""duration"": ""PT15M9S"", ""dimension"": ""2d"", ""de...","{""viewCount"": ""27"", ""likeCount"": ""5"", ""favorit..."
2,youtube#video,O5a6Idne7JJ0o5YTleqr4xT-LVw,Hd74vM8tTmI,"{""publishedAt"": ""2025-05-30T19:00:24Z"", ""chann...","{""duration"": ""PT31M14S"", ""dimension"": ""2d"", ""d...","{""viewCount"": ""19"", ""likeCount"": ""3"", ""favorit..."
3,youtube#video,rjFP7L6GaGo6Gxi0CFvy4O3Ip6M,Gq9SZWmkENg,"{""publishedAt"": ""2025-03-08T19:00:11Z"", ""chann...","{""duration"": ""PT32M35S"", ""dimension"": ""2d"", ""d...","{""viewCount"": ""50"", ""likeCount"": ""5"", ""favorit..."
4,youtube#video,WaFZE35Ow75MNZnxk_qx_yLhJYM,838e8voBnFw,"{""publishedAt"": ""2025-02-22T19:00:04Z"", ""chann...","{""duration"": ""PT3M20S"", ""dimension"": ""2d"", ""de...","{""viewCount"": ""152"", ""likeCount"": ""8"", ""favori..."
...,...,...,...,...,...,...
256,youtube#video,Eg1zqzrp0deHLeUW3WUDwUA3mAY,Xtgl91SUqNg,"{""publishedAt"": ""2015-04-16T08:08:45Z"", ""chann...","{""duration"": ""PT1M18S"", ""dimension"": ""2d"", ""de...","{""viewCount"": ""32"", ""likeCount"": ""2"", ""favorit..."
257,youtube#video,nl00_B-cWvJolBYOqws1-ekwsTY,d3xQvP8oLJk,"{""publishedAt"": ""2015-04-16T07:51:54Z"", ""chann...","{""duration"": ""PT1M19S"", ""dimension"": ""2d"", ""de...","{""viewCount"": ""21"", ""likeCount"": ""0"", ""favorit..."
258,youtube#video,Te57rper_I2W72ULm0-7cBkkmvY,zsnROJrCdG0,"{""publishedAt"": ""2015-04-13T04:36:33Z"", ""chann...","{""duration"": ""PT2M40S"", ""dimension"": ""2d"", ""de...","{""viewCount"": ""434"", ""likeCount"": ""3"", ""favori..."
259,youtube#video,elACgksnyZTI1_wOjJC1BJsuZYA,l51CZtRUnFo,"{""publishedAt"": ""2015-03-26T05:07:08Z"", ""chann...","{""duration"": ""PT1M47S"", ""dimension"": ""2d"", ""de...","{""viewCount"": ""271"", ""likeCount"": ""4"", ""favori..."


In [48]:
# Сохраняем в CSV при необходимости
#df.to_csv("youtube_channel_videos_matroskin.json.csv", index=False)